In [ ]:
import simpy
import numpy as np
import pandas as pd

"""
TITEL: Krankenhaussimulation zur Identifikation von Kapazitätsgrenzen (Stress-Test)

BESCHREIBUNG:
Dieses Skript führt eine diskrete Ereignissimulation (DES) durch, um die Belastbarkeit 
einer Krankenhausabteilung zu testen. Es kombiniert SimPy (Prozesslogik) mit 
Monte-Carlo-Methoden (statistische Absicherung).

ZIEL:
Herauszufinden, ab welcher Patientenanzahl pro Woche das System "kippt" (von stabil zu überlastet).
Dazu wird die Patientenlast schrittweise erhöht (Szenario-Analyse).
"""

# =========================================================
# 1. KONFIGURATION
# =========================================================
TOTAL_BEDS = 447            # Maximale Bettenkapazität (Ressource)
BASE_SEED = 42              # Startwert für den Zufallszahlengenerator (für Reproduzierbarkeit)
R = 20                      # Anzahl der Monte-Carlo-Läufe pro Szenario (zur Glättung von Zufallsschwankungen)
SIM_DURATION = 150          # Dauer einer einzelnen Simulation in Tagen
WARMUP_OCCUPANCY = 0.75     # Initiale Auslastung (75%), um "leeres System"-Bias zu vermeiden
MEAN_LOS = 14               # Mittlere Verweildauer (Length of Stay) in Tagen (Exponentialverteilt)

# =========================================================
# 2. PATIENTENPROZESS (DES)
# =========================================================
def patient_process(env, hospital, mean_los, rng, stats, is_preloaded=False):
    """
    Simuliert den Lebenszyklus eines einzelnen Patienten im Krankenhaus.
    
    Ablauf:
    1. Ankunft (Request eines Bettes).
    2. Warten (falls alle Betten belegt sind).
    3. Belegung (Start Service).
    4. Aufenthalt (Timeout für Dauer LOS).
    5. Entlassung (Release des Bettes).

    Args:
        env: SimPy Environment.
        hospital: SimPy Resource (Betten).
        mean_los: Durchschnittliche Aufenthaltsdauer.
        rng: Numpy Random Generator Instanz.
        stats: Dictionary zum Sammeln von Metriken.
        is_preloaded (bool): Wenn True, zählt der Patient nicht zur Wartezeit-Statistik 
                             (da er Teil des Warm-ups ist).
    """
    arrival_time = env.now

    # Anfrage an die Ressource 'Krankenhausbett'
    with hospital.request() as req:
        yield req  # Warten, bis ein Bett frei wird
        
        start_service = env.now

        # Nur "echte" neue Patienten zählen für die Wartezeit-Analyse.
        # Preload-Patienten sind schon "im Bett", wenn die Sim startet.
        if not is_preloaded:
            stats["wait_times"].append(start_service - arrival_time)

        # Patient belegt das Bett für eine zufällige Zeit (Exponentialverteilung)
        yield env.timeout(rng.exponential(mean_los))

# =========================================================
# 3. KRANKENHAUS-SETUP (INKL. WARM-UP)
# =========================================================
def setup_hospital(env, hospital, rng, stats, patients_per_week, mean_los):
    """
    Initialisiert das Krankenhaus und steuert den Patientenstrom.
    
    Funktionen:
    1. Warm-up: Füllt das Krankenhaus sofort zu Beginn zu X% (hier 75%), 
       damit die Simulation nicht bei 0 Bettenbelegung startet (realistischerer Startzustand).
    2. Generator: Erzeugt kontinuierlich neue Patienten basierend auf der Wochenrate.

    Args:
        env (simpy.Environment): Die aktive Simulationsumgebung.
        hospital (simpy.Resource): Die Bettenressource.
        rng (numpy.random.Generator): Zufallszahlengenerator.
        stats (dict): Sammelbehälter für Simulationsdaten.
        patients_per_week (int): Die Last-Variable für das aktuelle Szenario.
        mean_los (float): Durchschnittliche Aufenthaltsdauer.
    """
    # Umrechnung: Patienten pro Woche -> Durchschnittlicher Zeitabstand zwischen Ankünften (in Tagen)
    mean_interarrival = 7.0 / patients_per_week

    # 1. WARM-UP PHASE (System vorbefüllen)
    preload = int(TOTAL_BEDS * WARMUP_OCCUPANCY)
    for _ in range(preload):
        env.process(patient_process(
            env, hospital, mean_los, rng, stats, is_preloaded=True
        ))

    # 2. LAUFENDER BETRIEB (Generator-Schleife)
    while True:
        # Warten bis zum nächsten Patienten (zufällig, exponentialverteilt)
        yield env.timeout(rng.exponential(mean_interarrival))
        # Neuen Patientenprozess starten
        env.process(patient_process(
            env, hospital, mean_los, rng, stats
        ))

# =========================================================
# 4. ENGPASS-MONITOR (ZIELEREIGNIS Yi)
# =========================================================
def monitor_system(env, hospital, stats):
    """
    Beobachter-Prozess, der parallel läuft und den Systemzustand prüft.
    
    Zweck:
    Erfassung von "Soft-Metriken", die SimPy nicht automatisch speichert.
    Hier definiert: Was ist ein "Engpass"?
    
    Definition Engpass:
    - Es existiert eine Warteschlange (queue > 0)
      ODER
    - Alle Betten sind voll (count == capacity)
    
    Die Prüfung erfolgt alle 0.1 Tage (diskretisiert), um die Dauer des Engpasses
    (Time in Congestion) aufzusummieren.

    Args:
        env (simpy.Environment): Die aktive Simulationsumgebung.
        hospital (simpy.Resource): Die zu überwachende Ressource.
        stats (dict): Dictionary, in dem 'engpass_time' und 'engpass_flag' aktualisiert werden.
    """
    last_time = env.now

    while True:
        yield env.timeout(0.1)  # Prüf-Intervall (hohe Auflösung)
        now = env.now
        dt = now - last_time    # Vergangene Zeit seit letztem Check
        last_time = now

        queue_len = len(hospital.queue)
        occupancy = hospital.count

        # Prüfung auf Engpass-Kriterium
        if queue_len > 0 or occupancy >= TOTAL_BEDS:
            stats["engpass_time"] += dt  # Zeit im roten Bereich aufaddieren
            stats["engpass_flag"] = 1    # Markierung: In diesem Lauf gab es mind. einen Engpass

# =========================================================
# 5. EIN SIMULATIONSLAUF
# =========================================================
def run_single_simulation(patients_per_week, mean_los, seed):
    """
    Kapselt einen kompletten Simulationsdurchlauf (Replikation).
    
    Ablauf:
    1. Erstellt Environment und Ressource.
    2. Startet Setup (Patientengenerator) und Monitor.
    3. Führt Simulation bis SIM_DURATION aus.
    4. Gibt Rohdaten (stats) zurück.

    Args:
        patients_per_week (int): Anzahl Patientenankünfte pro Woche.
        mean_los (float): Mittlere Verweildauer in Tagen.
        seed (int): Startwert für den Zufall (wichtig für Reproduzierbarkeit).

    Returns:
        dict: Die gesammelten Statistiken (wait_times, engpass_time, engpass_flag).
    """
    rng = np.random.default_rng(seed)
    env = simpy.Environment()
    hospital = simpy.Resource(env, capacity=TOTAL_BEDS)

    # Container für Ergebnisse dieses einen Laufs
    stats = {
        "wait_times": [],
        "engpass_time": 0.0,
        "engpass_flag": 0
    }

    env.process(setup_hospital(
        env, hospital, rng, stats, patients_per_week, mean_los
    ))
    env.process(monitor_system(env, hospital, stats))

    env.run(until=SIM_DURATION)
    return stats

# =========================================================
# 6. MONTE-CARLO-SCHLEIFE
# =========================================================
def monte_carlo(patients_per_week, mean_los):
    """
    Führt R Replikationen für EIN bestimmtes Last-Szenario durch.
    
    Zweck:
    Da Simulationen stochastisch sind (Zufall), reicht ein Lauf nicht aus.
    Wir mitteln über R Läufe, um statistisch signifikante Aussagen zu treffen.
    
    Rückgabe:
    - Mittelwert & Konfidenzintervall der Wartezeit.
    - Engpass-Wahrscheinlichkeit (Wie viel % der R Läufe hatten Probleme?).
    - Durchschnittliche Zeit im Engpass.

    Args:
        patients_per_week (int): Die zu testende Last im aktuellen Szenario.
        mean_los (float): Mittlere Verweildauer.

    Rückgabe als tuple was enthält:
    - Mittelwert & Konfidenzintervall der Wartezeit.
    - Engpass-Wahrscheinlichkeit (Wie viel % der R Läufe hatten Probleme?).
    - Durchschnittliche Zeit im Engpass.
    """
    wait_means = []
    engpass_flags = []
    engpass_times = []

    for i in range(R):
        # Jeder Lauf bekommt einen eigenen Seed (BASE_SEED + i), damit sie unterschiedlich sind
        stats = run_single_simulation(
            patients_per_week, mean_los, BASE_SEED + i
        )

        if stats["wait_times"]:
            wait_means.append(np.mean(stats["wait_times"]))

        engpass_flags.append(stats["engpass_flag"])
        engpass_times.append(stats["engpass_time"] / SIM_DURATION) # Normierung auf % der Zeit

    # Berechnung der Statistik über alle R Läufe
    if wait_means:
        mean_wait = np.mean(wait_means)
        se_wait = np.std(wait_means, ddof=1) / np.sqrt(R) # Standardfehler
        ci_wait = (mean_wait - 1.96 * se_wait, mean_wait + 1.96 * se_wait) # 95% KI
    else:
        mean_wait, se_wait, ci_wait = 0, 0, (0, 0)

    p_engpass = np.mean(engpass_flags)       # z.B. 0.4 bedeutet: in 40% der Läufe gab es Stau
    mean_engpass_time = np.mean(engpass_times)

    return mean_wait, se_wait, ci_wait, p_engpass, mean_engpass_time

# =========================================================
# 7. LASTSTEIGERUNGS-SZENARIO (5er-Schritte)
# =========================================================
"""
Hier findet der eigentliche "Stress-Test" statt.
Wir iterieren von einer niedrigen Last (35 Pat/Woche) bis zu einer 
sehr hohen Last (235 Pat/Woche) in 5er Schritten.

Dies ermöglicht die Erstellung einer Kennlinie:
X-Achse: Patientenaufkommen
Y-Achse: Wahrscheinlichkeit eines Engpasses

So lässt sich der "Kipppunkt" (Tipping Point) visuell oder rechnerisch finden.
"""
patients_range = range(35, 236, 5)  # Start=35, Ende<236, Schrittweite=5
results = []

for p_week in patients_range:
    # Monte Carlo Analyse für diesen spezifischen Last-Punkt
    m, se, ci, p_e, t_e = monte_carlo(p_week, MEAN_LOS)
    
    # Theoretische Auslastung (Rho) nach Formel: (Lambda * Mu) / Kapazität
    # Dient als Referenzwert zur Simulation.
    rho = ((p_week / 7) * MEAN_LOS) / TOTAL_BEDS

    results.append({
        "Pat./Woche": p_week,
        "Auslastung ρ": rho,
        "Ø Wartezeit (d)": m,
        "SE": se,
        "95%-KI": f"[{ci[0]:.2f}, {ci[1]:.2f}]",
        "P(Engpass)": p_e,              # Kritische Metrik 1: Risiko
        "Zeit im Engpass (%)": t_e * 100 # Kritische Metrik 2: Schweregrad
    })

# =========================================================
# 8. OUTPUT & SCHWELLENIDENTIFIKATION
# =========================================================
df = pd.DataFrame(results)

# Kategorisierung für schnelle Lesbarkeit
df["Belastungsstatus"] = df["P(Engpass)"].apply(
    lambda x: "OK" if x < 0.1 else             # < 10% Risiko
              "kritisch" if x < 0.5 else       # 10-50% Risiko
              "Überlastet"                     # > 50% Risiko
)

print("\nLASTSTEIGERUNGS-ANALYSE (DES + MONTE CARLO)")
print("=" * 120)
# Formatierte Ausgabe der Tabelle
print(df.to_string(index=False, formatters={
    "Auslastung ρ": lambda x: f"{x*100:.1f}%",
    "Ø Wartezeit (d)": lambda x: f"{x:.3f}",
    "SE": lambda x: f"{x:.3f}",
    "P(Engpass)": lambda x: f"{x:.2f}",
    "Zeit im Engpass (%)": lambda x: f"{x:.1f}"
}))
print("=" * 120)

# Automatische Identifikation des Kipppunkts
# Wir definieren den Kipppunkt hier als den Moment, wo das Engpass-Risiko 50% überschreitet.
critical = df[df["P(Engpass)"] >= 0.5]

if not critical.empty:
    first = critical.iloc[0]
    print(
        f"\n⚠️ Belastungsschwelle ab ca. "
        f"{first['Pat./Woche']} Patienten/Woche "
        f"(P(Engpass) ≥ 50%)"
    )
else:
    print("\n✅ Keine kritische Belastung im untersuchten Bereich.")

print(
    f"\nParameter: Betten={TOTAL_BEDS}, LOS={MEAN_LOS} Tage, "
    f"R={R}, Dauer={SIM_DURATION} Tage"
)